In [1]:
import os
from os.path import *

In [2]:
validation='/media/Gui2/thermix/ARTraining/trained_models/thermix_42a/thermix_42a_testing.txt'
'''
if !predictions, run:
    go
    cd ccv/bin
    ./cnnclassify /media/Gui2/thermix/ARTraining/trained_models/thermix_42a/thermix_42a_testing.txt \
                    /media/Gui2/thermix/ARTraining/trained_models/thermix_42a/thermix_42a.sqlite3 \
                    /media/Gui2/thermix/dataset_merged \
                    | tee /media/Gui2/thermix/ARTraining/trained_models/thermix_42a/thermix_42a_testing_predictions.txt
'''
predictions='/media/Gui2/thermix/ARTraining/trained_models/thermix_42a/thermix_42a_testing_predictions.txt'

In [3]:
def get_validation_vs_prediction():
    
    val_file = open(validation, 'r')
    val_lines = val_file.readlines()
    val_file.close()

    pred_file = open(predictions, 'r')
    pred_lines = pred_file.readlines()
    pred_file.close()
        
    vals = []
    for line in val_lines:
        c, p = line.rstrip().split(' ')
        vals.append([c, p])
        
    preds = []
    for line in pred_lines:
        p, c, s = line.rstrip().split(' ')
        preds.append([c, p, s])
        
    assert len(vals) == len(preds)
    
    all_data = []
    for v in vals:
        for p in preds:
            if v[1] in p:
                all_data.append([v[1], v[0], p[0], p[2]])
    
    # all_data[i] = [<path_i>, <real_class_i>, <predicted_class_i>, <prediction_score_i>]
    return all_data

In [4]:
def overall_accuracy(population):
    tp = []
    for y, h in population:
        if y == h:
            tp.append([y,h])
    
    accuracy = count(tp)/count(population)
    
    print '* class by class accuracy: %.2f %%' %(accuracy*100)
    
#def overall_miss_rate(population):
#    fn = [_,0,0,0,0,0]
#    population_classes = [_,0,0,0,0,0]
#    for y, h in population:
#        population_classes[y] = population_classes[y] + 1
#        if y != h:
#            fn[y] = fn[y] + 1
#            
#    miss_rate = 0            
#    miss_rate = sum([fn[i]/population_classes[i] for i in range(1, len(population_classes))])
#    print '* class by class miss_rate: %.2f %%' %(miss_rate*100)

In [5]:
def metrics(population):
    print 'positive => lying, sitting, standing, people (interesting)'
    print 'negative => background (not interesting)'
    print '-----'
    
    tp = []
    tn = []
    fp = []
    fn = []

    condition_positive = [[y,h] for [y,h] in data if is_positive(y)]
    condition_negative = [[y,h] for [y,h] in data if is_negative(y)]
    
    assert(len(population) == len(condition_negative)+len(condition_positive))
    
    for y,h in population:
        # true positive => y == 1,2,3,4 and h == 1,2,3,4
        if is_positive(y) and is_positive(h):
            tp.append([y,h])
            
        # get true negatives
        elif is_negative(y) and is_negative(h):
            tn.append([y,h])
            
        # get false positives => y == 5 and h == 1,2,3,4
        elif is_negative(y) and is_positive(h):
            fp.append([y,h])
            
        # get false negatives
        elif is_positive(y) and is_negative(h):
            fn.append([y,h])
            
    assert(len(tp)+len(tn)+len(fp)+len(fn) == len(population))
    
    # sensitivity = true positives / real positives
    sensitivity = count(tp)/count(condition_positive)
    
    # specificity = true negatives / real negatives
    specificity = count(tn)/count(condition_negative)
    
    # precision = true positives / total of correct predictions
    precision = count(tp)/(count(tp)+count(fp))
    
    # negative predictive value = true negatives / total of correct predictions
    negative_predictive_value = count(tn)/(count(tn)+count(fn))
    
    # accuracy = correct predicitons / size of population
    accuracy = (count(tp)+count(tn))/(count(population))
    
    print '* accuracy: %.2f %%' %(accuracy*100)
    print '* sensitivity (recall, true positive rate): %.2f %%' %(sensitivity*100)
    print '* specificity (true negative rate): %.2f %%' %(specificity*100)
    print '* precision (positive predictive value) : %.2f %%' %(precision*100)
    print '* negative predictive value: %.2f %%' %(negative_predictive_value*100)
    print '* fall-out (false positive rate): %.2f %%' %((1-specificity)*100)
    print '* miss-rate (false negative rate): %.2f %%' %((1-sensitivity)*100)
    print '* false-discovery-rate: %.2f %%' %((1-precision)*100)
    
    
def is_positive(x):
    return (x == 1 or x == 2 or x == 3 or x == 4)

def is_negative(x):
    return (x == 5)

def count(arr):
    return float(len(arr))

In [6]:
%matplotlib inline
import matplotlib
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

classes = ['lying', 'sitting', 'standing', 'people', 'background']
def get_confusion_matrix(data):
    # all_data[i] = [<path_i>, <real_class_i>, <predicted_class_i>, <prediction_score_i>]
    y_true = [y for y,_ in data]
    y_pred = [y for _,y in data]
    cm = confusion_matrix(y_true, y_pred)
    np.set_printoptions(precision=2)
    
#    plt.figure()
#    plot_confusion_matrix(cm)
#    print(cm)
#    plt.show()
    
    # Normalize the confusion matrix by row (i.e by the number of samples
    # in each class)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm_normalized)
    
    #plt.figure()
    #plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')
    #plt.show()

    plotly_confusion_matrix(cm_normalized)


In [7]:
'''plots in jupyter the confusion matrix'''
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.tight_layout()
    plt.ylabel('y')
    plt.xlabel('h(x)')

In [8]:
import plotly.plotly as py
import plotly.graph_objs as go
def plotly_confusion_matrix(cm, title='Confusion matrix '+os.path.basename(validation)):
    annotations = []
    for n, row in enumerate(cm):
        for m, val in enumerate(row):
            var = cm[n][m]
            annotations.append(
                dict(
                    text=str(truncate(val,3)),
                    x=classes[m], y=classes[n],
                    xref='x1', yref='y1',
                    font=dict(color='white'),
                    showarrow=False)
                )
    data = [
        go.Heatmap(
            x=classes,
            y=classes,
            z=cm,
#            colorscale='Viridis'
        )
    ]
    
    layout = go.Layout(
            title='Confusion Matrix',
            annotations=annotations,
            xaxis=dict(title='Predicted value',),
            yaxis=dict(title='Real value',)
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename=os.path.basename(validation)+' confusion matrix')
    
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '{}'.format(f)
    if 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [10]:
print 'validation: '+os.path.basename(validation)
print 'prediction: '+os.path.basename(predictions)
print '-----'
data = [[int(y),int(h)] for _,y,h,_ in get_validation_vs_prediction()]
metrics(data)
overall_accuracy(data)
get_confusion_matrix(data)

#from sklearn.metrics import accuracy_score
#print accuracy_score([y for _,y,_,_ in get_validation_vs_prediction()], [y for _,_,y,_ in get_validation_vs_prediction()])

validation: thermix_42a_testing.txt
prediction: thermix_42a_testing_predictions.txt
-----
positive => lying, sitting, standing, people (interesting)
negative => background (not interesting)
-----
* accuracy: 98.20 %
* sensitivity (recall, true positive rate): 98.97 %
* specificity (true negative rate): 95.05 %
* precision (positive predictive value) : 98.78 %
* negative predictive value: 95.81 %
* fall-out (false positive rate): 4.95 %
* miss-rate (false negative rate): 1.03 %
* false-discovery-rate: 1.22 %
* class by class accuracy: 91.91 %
[[  9.68e-01   3.96e-03   0.00e+00   0.00e+00   2.77e-02]
 [  8.38e-02   8.85e-01   8.88e-03   2.27e-02   0.00e+00]
 [  1.48e-02   1.09e-02   9.42e-01   1.98e-02   1.29e-02]
 [  7.55e-02   3.21e-02   3.78e-02   8.54e-01   9.44e-04]
 [  4.75e-02   0.00e+00   1.98e-03   0.00e+00   9.51e-01]]


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.

